In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np


In [ ]:
df_temp = pd.read_csv("monthly-mean-temp.csv")
df_temp['Month'] = pd.to_datetime(df_temp['Month'])
df_temp.set_index('Month', inplace=True)
df_temp.head()


In [ ]:
plt.figure(figsize=(14, 6))
plt.plot(df_temp.index, df_temp['Temperature'], color='tab:blue', linewidth=1.5)
plt.title('Temperatura Media Mensual (1920 - 1939)')
plt.xlabel('Fecha')
plt.ylabel('Temperatura (°F)')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
df_temp['SMA_3'] = df_temp['Temperature'].rolling(window=3).mean()
df_temp['SMA_12'] = df_temp['Temperature'].rolling(window=12).mean()

plt.figure(figsize=(14, 6))
plt.plot(df_temp['Temperature'], label='Original', linewidth=1.5)
plt.plot(df_temp['SMA_3'], label='Media móvil 3 meses', linestyle='--')
plt.plot(df_temp['SMA_12'], label='Media móvil 12 meses', linestyle='--')
plt.title('Promedios Móviles - Temperatura Media Mensual')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
df_temp['SES'] = df_temp['Temperature'].ewm(alpha=0.3, adjust=False).mean()

plt.figure(figsize=(14, 6))
plt.plot(df_temp['Temperature'], label='Original', linewidth=1.5)
plt.plot(df_temp['SES'], label='Alisamiento Exponencial Simple (α=0.3)', linestyle='--')
plt.title('Alisamiento Exponencial Simple - Temperatura')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
modelo_hw_temp = ExponentialSmoothing(df_temp['Temperature'], trend='add', seasonal='add', seasonal_periods=12).fit()
df_temp['HW_add'] = modelo_hw_temp.fittedvalues

plt.figure(figsize=(14, 6))
plt.plot(df_temp['Temperature'], label='Original', linewidth=1.5)
plt.plot(df_temp['HW_add'], label='Holt-Winters Aditivo', linestyle='--')
plt.title('Modelo Holt-Winters Aditivo - Temperatura')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
from statsmodels.tsa.stattools import adfuller
adf_temp = adfuller(df_temp['Temperature'].dropna())
print("ADF Statistic:", adf_temp[0])
print("p-value:", adf_temp[1])
print("Critical Values:", adf_temp[4])


In [ ]:
df_temp['Temp_diff'] = df_temp['Temperature'].diff()
adf_temp_diff = adfuller(df_temp['Temp_diff'].dropna())
print("ADF Statistic (diferenciada):", adf_temp_diff[0])
print("p-value:", adf_temp_diff[1])
print("Critical Values:", adf_temp_diff[4])


In [ ]:
sarima_temp = SARIMAX(df_temp['Temperature'], order=(1,1,1), seasonal_order=(1,1,1,12)).fit(disp=False)
df_temp['SARIMA_pred'] = sarima_temp.fittedvalues

plt.figure(figsize=(14, 6))
plt.plot(df_temp['Temperature'], label='Original', linewidth=1.5)
plt.plot(df_temp['SARIMA_pred'], label='SARIMA (1,1,1)(1,1,1,12)', linestyle='--')
plt.title('Modelo SARIMA - Temperatura Media Mensual')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
original_temp = df_temp['Temperature'].dropna()
modelos_temp = {
    'SES': df_temp['SES'],
    'Holt-Winters': df_temp['HW_add'],
    'SARIMA': df_temp['SARIMA_pred']
}

resultados_temp = []
for name, pred in modelos_temp.items():
    aligned = original_temp.align(pred, join='inner')
    mae = mean_absolute_error(aligned[0], aligned[1])
    rmse = mean_squared_error(aligned[0], aligned[1], squared=False)
    resultados_temp.append((name, mae, rmse))

df_resultados_temp = pd.DataFrame(resultados_temp, columns=['Modelo', 'MAE', 'RMSE'])
df_resultados_temp
